In [1]:
import numpy as np
import pandas as pd
import requests as r
from bs4 import BeautifulSoup
from time import sleep
from random import randint
from IPython.core.display import clear_output
import time
from warnings import warn

In [2]:
#getting the total number of titles released in a given year
def get_number_of_titles(url):
    response = r.get(url)
    html_soup = BeautifulSoup(response.text, 'html.parser')
    titles = html_soup.find_all('div', class_ = 'desc')
    total_of_titles = int(titles[0].text.split()[2].replace(',', ''))
    return total_of_titles

In [3]:
#getting the cast members and directors of the movie
def get_cast(first_movie):
    director = ""
    s = first_movie.find_all('p', class_ = "")[0].text
    "".join(s.split())
    l = s.split()
    director = l[1] + " " + l[2]
    stars_string = l[5:]
    stars = []
    for i in range(0,len(stars_string)-1, 2):
        stars.append(stars_string[i] + " " + stars_string[i+1])
    stars = tuple(stars)
    return director, stars

In [4]:
years_url = [str(i) for i in range(1990,2011)]

In [5]:
url = 'https://www.imdb.com/search/title/?release_date=2021&count=100&has=business-info&adult=include'

In [6]:
response = r.get(url)
page_html = BeautifulSoup(response.text, 'html.parser')
first_movie = page_html.find('div', class_ = 'lister-item mode-advanced')

In [7]:
a = first_movie.h3.find('a').get('href')[7:15]

In [8]:
a

'tt238232'

In [9]:
def imdb_scraping(year_movies):
    # Redeclaring the lists to store data in
    names = []
    years = []
    imdb_ratings = []
    metascores = []
    votes = []
    grosses = []
    certificates = []
    runtimes = []
    genres = []
    directors = []
    stars = []
    sinopses = []
    movie_ids = []

    # Preparing the monitoring of the loop
    start_time = time.time()
    requests = 0
    year = year_movies

    number = get_number_of_titles('https://www.imdb.com/search/title/?release_date=' + year + '&count=100&has=business-info&adult=include')
    iterations = number // 100

    if iterations > 72:
        iterations = 72

    # For every year in the interval 2000-2017
    for i in range(int(iterations)):
        # Make a get request
        start = 1
        string = f'{start}'
        response = r.get('https://www.imdb.com/search/title/?release_date=' + year + '&count=100&has=business-info&adult=include&start=' + string)

        # Pause the loop
        sleep(randint(8,15))

        # Monitor the requests
        requests += 1
        elapsed_time = time.time() - start_time
        print('Request:{}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
        clear_output(wait = True)

        # Throw a warning for non-200 status codes
        if response.status_code != 200:
            warn('Request: {}; Status code: {}'.format(requests, response.status_code))

        # Break the loop if the number of requests is greater than expected
        if requests > 72:
            warn('Number of requests was greater than expected.')
            break

        # Parse the content of the request with BeautifulSoup
        page_html = BeautifulSoup(response.text, 'html.parser')

        # Select all the 50 movie containers from a single page
        mv_containers = page_html.find_all('div', class_ = 'lister-item mode-advanced')

        # For every movie of these 50
        for container in mv_containers:
            # If the movie has a Metascore, then:
            if container.find('div', class_ = 'ratings-metascore') is not None:  
                # The name
                name = container.h3.a.text
                names.append(name)
                # The year
                year = container.h3.find('span', class_ = "lister-item-year text-muted unbold").text
                years.append(year)
                # The IMDB rating
                imdb = float(container.strong.text)
                imdb_ratings.append(imdb)
                
                movie_id = container.h3.find('a').get('href')[7:16]
                movie_ids.append(movie_id)
                # The Metascore
                
                m_score = container.find('span', class_ = 'metascore').text
                metascores.append(int(m_score))
                
                if container.find('span', class_ = 'certificate') is not None:
                    ct = container.find('span', class_ = 'certificate').text
                    certificates.append(ct)
                else:
                    certificates.append("null")
                
                if container.find('span', class_ = 'runtime') is not None:
                    rt = container.find('span', class_ = 'runtime').text
                    runtimes.append(rt)
                else:
                    runtimes.append("null")
            
                if container.find('span', class_ = 'genre') is not None:
                    gr = container.find('span', class_ = 'genre').text
                    genres.append(gr)
                else:
                    genres.append("null")
                
                sinopse = container.find_all('p', class_ = "text-muted")[1].text
                sinopses.append(sinopse)
                
                director, cast = get_cast(container)
                directors.append(director)
                stars.append(cast)
                
                # The number of votes
                vote = container.find_all('span', attrs = {'name':'nv'})[0]['data-value']
                votes.append(int(vote))
                if len(container.find_all('span', attrs = {'name':'nv'})) > 1:
                    gross = container.find_all('span', attrs = {'name':'nv'})[1]['data-value']
                    grosses.append(gross)
                else:
                    grosses.append("null")
            start += 100
            
    movie_ratings = pd.DataFrame({'movie_id': movie_ids,
    'movie': names,
    'year': years,
    'imdb': imdb_ratings,
    'metascore': metascores,
    'votes': votes,
    'gross':grosses,
    'certificate': certificates,
    'runtime': runtimes,
    'genre': genres,
    'director': directors,
    'cast': stars,
    'sinopse': sinopses})
            
    return movie_ratings

In [10]:
year = "2003"

In [11]:
dataframe = imdb_scraping(year)

Request:52; Frequency: 0.07604941817889928 requests/s


In [12]:
df_2003 = dataframe.copy()

In [16]:
df_2003.head(15)

,movie_id,movie,year,imdb,metascore,votes,gross,certificate,runtime,genre,director,cast,sinopse
0,tt0332379,Escola de Rock,(2003),7.1,82,290994,"81,261,177",Livre,109 min,"\nComedy, Music",Richard Linklater,"(Jack Black,, Mike White,, Joan Cusack,, Adam ...","\nAfter being kicked out of his rock band, Dew..."
1,tt0234215,Matrix Reloaded,(2003),7.2,62,542775,"281,492,479",12,138 min,"\nAction, Sci-Fi","Lana Wachowski,","(| Stars:, Keanu Reeves,, Laurence Fishburne,,...","\nFreedom fighters Neo, Trinity and Morpheus c..."
2,tt0364569,Oldboy,(2003),8.4,77,541635,"707,481",18,120 min,"\nAction, Drama, Mystery",Park Chan-Wook,"(Choi Min-sik,, Yoo Ji-Tae,, Kang Hye-jeong,, ...",\nAfter being kidnapped and imprisoned for fif...
3,tt0325980,Piratas do Caribe: A Maldição do Pérola Negra,(2003),8.0,63,1045996,"305,413,918",14,143 min,"\nAction, Adventure, Fantasy",Gore Verbinski,"(Johnny Depp,, Geoffrey Rush,, Orlando Bloom,,...",\nBlacksmith Will Turner teams up with eccentr...
4,tt0167260,O Senhor dos Anéis: O Retorno do Rei,(2003),8.9,94,1712097,"377,845,905",14,201 min,"\nAction, Adventure, Drama",Peter Jackson,"(Elijah Wood,, Viggo Mortensen,, Ian McKellen,...",\nGandalf and Aragorn lead the World of Men ag...
5,tt0242653,Matrix Revolutions,(2003),6.8,47,468958,"139,313,948",14,129 min,"\nAction, Sci-Fi","Lana Wachowski,","(| Stars:, Keanu Reeves,, Laurence Fishburne,,...",\nThe human city of Zion defends itself agains...
6,tt0266697,Kill Bill: Volume 1,(2003),8.1,69,1047392,"70,099,045",18,111 min,"\nAction, Crime, Drama",Quentin Tarantino,"(Uma Thurman,, David Carradine,, Daryl Hannah,...","\nAfter awakening from a four-year coma, a for..."
7,tt0314331,Simplesmente Amor,(2003),7.6,55,448122,"59,696,144",14,135 min,"\nComedy, Drama, Romance",Richard Curtis,"(Hugh Grant,, Martine McCutcheon,, Liam Neeson...",\nFollows the lives of eight very different co...
8,tt0309987,Os Sonhadores,(2003),7.2,62,116358,"2,531,462",16,115 min,"\nDrama, Romance",Bernardo Bertolucci,"(Michael Pitt,, Louis Garrel,, Eva Green,, Ann...",\nA young American studying in Paris in 1968 s...
9,tt0327056,Sobre Meninos e Lobos,(2003),7.9,84,437349,"90,135,191",16,138 min,"\nCrime, Drama, Mystery",Clint Eastwood,"(Sean Penn,, Tim Robbins,, Kevin Bacon,, Emmy ...",\nThe lives of three men who were childhood fr...


In [22]:
df_2003.loc[df_2003['movie'] == '+Velozes +Furiosos']

,movie_id,movie,year,imdb,metascore,votes,gross,certificate,runtime,genre,director,cast,sinopse
34,tt0322259,+Velozes +Furiosos,(2003),5.9,38,263212,"127,154,901",14,107 min,"\nAction, Crime, Thriller",John Singleton,"(Paul Walker,, Tyrese Gibson,, Cole Hauser,, E...",\nFormer cop Brian O'Conner is called upon to ...


In [512]:
movie_ratings.to_csv('movie_ratings.csv')